# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Read and Preprocess data

In [ ]:
data_df = pd.read_fwf("/content/german.data-numeric", header=None)
data_df = data_df.add_prefix("Attr_")
data_df.rename(columns={"Attr_24": "Target"}, inplace=True)
data_df

,Attr_0,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_15,Attr_16,Attr_17,Attr_18,Attr_19,Attr_20,Attr_21,Attr_22,Attr_23,Target
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,12,2,17,1,4,2,4,1,31,...,0,0,1,0,0,1,0,1,0,1
996,1,30,2,39,1,3,1,4,2,40,...,0,1,1,0,0,1,0,0,0,1
997,4,12,2,8,1,5,3,4,3,38,...,0,0,1,0,0,1,0,0,1,1
998,1,45,2,18,1,3,3,4,4,23,...,0,0,1,0,0,0,0,0,1,2


In [ ]:
# PARAMETERS
N_TEST_SAMPLES = 200
TARGET = 'Target'

# Split to Train-Validation and Test sets
train_val_df = data_df.iloc[:-N_TEST_SAMPLES, :]
test_df = data_df.iloc[-N_TEST_SAMPLES:, :]

# Scale data
scaler = MinMaxScaler()
train_val_sc = scaler.fit_transform(train_val_df)
train_val_df_sc = pd.DataFrame(train_val_sc, columns=train_val_df.columns, index=train_val_df.index)
test_sc = scaler.transform(test_df)
test_df_sc = pd.DataFrame(test_sc, columns=test_df.columns, index=test_df.index)


# split to features and targets
X_train_val_df, y_train_val_df = train_val_df_sc.drop(TARGET, axis=1), train_val_df_sc[TARGET]
X_test_df, y_test_df = test_df_sc.drop(TARGET, axis=1), test_df_sc[TARGET]

# Split to train and validation sets 
X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(X_train_val_df,
                                                              y_train_val_df, 
                                                              test_size=0.4,
                                                              shuffle=True,
                                                              random_state=0)

print(X_train_df.shape, X_val_df.shape, X_test_df.shape)
print(y_train_df.shape, y_val_df.shape, y_test_df.shape)

# Change 0's to -1's, because SVM requires that labels are +1 and -1
y_train_df[y_train_df == 0.] = -1
y_val_df[y_val_df == 0.] = -1
y_test_df[y_test_df == 0.] = -1

(480, 24) (320, 24) (200, 24)
(480,) (320,) (200,)


# 1. Minimizing the expected cost

In [ ]:
# create the cost matrix
cost_matrix = [[0, 1],
               [5, 0]]

classifiers = {
    'random forest': RandomForestClassifier(n_estimators=100, random_state=0),
    'linear SVM': SVC(kernel='linear', probability=True),
    'naive bayes': GaussianNB(),
}

y_test = y_test_df.to_numpy(dtype='int64')

for name in classifiers.keys():
  base_clf = classifiers[name]
  print(f'======================== {name} ======================== \n')

  print(" ***** Cost minimization WITHOUT probability calibration *****")
  base_clf.fit(pd.concat([X_train_df, X_val_df]), 
               pd.concat([y_train_df, y_val_df]))
  y_pred_probs = base_clf.predict_proba(X_test_df)
  y_preds = np.argmin(np.matmul(y_pred_probs, np.array(cost_matrix).T), axis=1)
  y_preds[y_preds == 0] = -1
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

  print(" ***** Cost minimization with SIGMOID calibration *****")
  base_clf.fit(X_train_df, y_train_df)
  calibrated_clf = CalibratedClassifierCV(base_clf, method="sigmoid", cv='prefit')
  calibrated_clf.fit(X_val_df, y_val_df)
  y_pred_probs = calibrated_clf.predict_proba(X_test_df)
  y_preds = np.argmin(np.matmul(y_pred_probs, np.array(cost_matrix).T), axis=1)
  y_preds[y_preds == 0] = -1
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

  print(" ***** Cost minimization with ISOTONIC calibration *****")
  base_clf.fit(X_train_df, y_train_df)
  calibrated_clf = CalibratedClassifierCV(base_clf, method="isotonic", cv='prefit')
  calibrated_clf.fit(X_val_df, y_val_df)
  y_pred_probs = calibrated_clf.predict_proba(X_test_df)
  y_preds = np.argmin(np.matmul(y_pred_probs, np.array(cost_matrix).T), axis=1)
  y_preds[y_preds == 0] = -1
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"Total Cost = {total_cost}")

======================== random forest ======================== 

 ***** Cost minimization WITHOUT probability calibration *****


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1       0.69      1.00      0.82       139
           1       0.00      0.00      0.00        61

    accuracy                           0.69       200
   macro avg       0.35      0.50      0.41       200
weighted avg       0.48      0.69      0.57       200

[[139  61]
 [  0   0]]

Total Cost = 61
 ***** Cost minimization with SIGMOID calibration *****


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1       0.69      1.00      0.82       139
           1       0.00      0.00      0.00        61

    accuracy                           0.69       200
   macro avg       0.35      0.50      0.41       200
weighted avg       0.48      0.69      0.57       200

[[139  61]
 [  0   0]]

Total Cost = 61
 ***** Cost minimization with ISOTONIC calibration *****
              precision    recall  f1-score   support

          -1       0.72      1.00      0.84       139
           1       1.00      0.11      0.21        61

    accuracy                           0.73       200
   macro avg       0.86      0.56      0.52       200
weighted avg       0.81      0.73      0.64       200

[[139  54]
 [  0   7]]
Total Cost = 54
======================== linear SVM ======================== 

 ***** Cost minimization WITHOUT probability calibration *****
              precision    recall  f1-score   support

          -1       0.71      

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

Από τα παραπάνω αποτελέσματα παρατηρώ τα παρακάτω:
1. Για το **Random Forest**:
  * *χωρίς probability calibration* έχουμε τα χειρότερα αποτελέσματα με **total cost = 61**.
  * με *sigmoid calibration* έχουμε τα ίδια αποτελέσματα σε σχέση με την προηγούμενη περίπτωση.
  * με *isotonic calibration* πετυχαίνουμε τα καλύτερα αποτελέσματα για τον συγκεκριμένο αλγόριθμο με **total cost = 54**.

2. Για το **Linear SVM**:
  * *χωρίς probability calibration* έχουμε τα χειρότερα αποτελέσματα με **total cost = 67**.
  * με *sigmoid calibration* πετυχαίνουμε τα καλύτερα αποτελέσματα για τον συγκεκριμένο αλγόριθμο με **total cost = 60**.
  * με *isotonic calibration* πετυχαίνουμε παραπλήσια αποτελέσματα, σε σχέση με την περίπτωση του sigmoid calibration, με **total cost = 60**.

3. Για το **Naive Bayes**:
  * *χωρίς probability calibration* έχουμε τα χειρότερα αποτελέσματα με **total cost = 120**.
  * με *sigmoid calibration* πετυχαίνουμε τα καλύτερα αποτελέσματα για τον συγκεκριμένο αλγόριθμο με **total cost = 61**.
  * με *isotonic calibration* πετυχαίνουμε τα ίδια αποτελέσματα σε σχέση με την περίπτωση του sigmoid calibration με **total cost = 61**.

Συνολικά, παρατηρούμε ότι:
* στην προκειμένη περίπτωση **η μέθοδος probability calibration (τόσο η sigmoid όοο και η isotonic) *βελτιώνουν* την επίδοση των παραπάνω αλγορίθμων**
* στο συγκεκειμένο task και dataset, τα **καλύτερα αποτελέσματα** πετυχαίνει το μοντέλο **Random Forest**.
* στο συγκεκειμένο task και dataset, τη **μεγαλύτερη βελτίωση όταν εφαρμόζεται calibration** πετυχαίνει το μοντέλο **Naive Bayes**.

# 2. Sampling

In [ ]:
X_train_val_df = pd.concat([X_train_df, X_val_df])
y_train_val_df = pd.concat([y_train_df, y_val_df])

In [ ]:
y_train_val_df.value_counts()

-1.0    561
 1.0    239
Name: Target, dtype: int64

In [ ]:
# create the cost matrix
cost_matrix = [[0, 1],
               [5, 0]]

classifiers = {
    'random forest': RandomForestClassifier(n_estimators=100, random_state=0),
    'linear SVM': SVC(kernel='linear', probability=True),
    'naive bayes': GaussianNB(),
}

y_test = y_test_df.to_numpy(dtype='int64')

for name in classifiers.keys():
  base_clf = classifiers[name]
  print(f'======================== {name} ========================')

  print("\n***** WITHOUT sampling *****")
  base_clf.fit(X_train_val_df, y_train_val_df)
  y_preds = base_clf.predict(X_test_df)
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

  print("\n***** with UNDER-SAMPLING *****")
  sampler = RandomUnderSampler(sampling_strategy={-1: 239, 1: 239}, random_state=0)
  X_rs_df, y_rs_df = sampler.fit_resample(X_train_val_df, y_train_val_df)
  print(y_rs_df.value_counts())
  base_clf.fit(X_rs_df, y_rs_df)
  y_preds = base_clf.predict(X_test_df)
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

  print("\n***** with OVER-SAMPLING *****")
  sampler = RandomOverSampler(sampling_strategy={-1: 561, 1: 561}, random_state=0)
  X_rs_df, y_rs_df = sampler.fit_resample(X_train_val_df, y_train_val_df)
  print(y_rs_df.value_counts())
  base_clf.fit(X_rs_df, y_rs_df)
  y_preds = base_clf.predict(X_test_df)
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

======================== random forest ========================

***** WITHOUT sampling *****
              precision    recall  f1-score   support

          -1       0.78      0.92      0.84       139
           1       0.69      0.39      0.50        61

    accuracy                           0.76       200
   macro avg       0.73      0.66      0.67       200
weighted avg       0.75      0.76      0.74       200

[[128  37]
 [ 11  24]]

Total Cost = 92

***** with UNDER-SAMPLING *****
-1.0    239
 1.0    239
Name: Target, dtype: int64
              precision    recall  f1-score   support

          -1       0.85      0.76      0.81       139
           1       0.57      0.70      0.63        61

    accuracy                           0.74       200
   macro avg       0.71      0.73      0.72       200
weighted avg       0.77      0.74      0.75       200

[[106  18]
 [ 33  43]]

Total Cost = 183

***** with OVER-SAMPLING *****
 1.0    561
-1.0    561
Name: Target, dtype: int64
    

Από τα παραπάνω αποτελέσματα παρατηρώ τα παρακάτω:
1. Για το **Random Forest**:
  * *χωρίς sampling* έχουμε τα καλύτερα αποτελέσματα με **total cost = 92**.
  * με *undersampling* έχουμε τα χειρότερα αποτελέσματα για τον συγκεκριμένο αλγόριρθμο.
  * με *οversampling* επίσης πετυχαίνουμε χειρότερα αποτελέσματα σε σχέση με την περίπτωση που δεν εφαρμόζουμε καθόλου την τεχνική του sampling.

2. Για το **Linear SVM**:
  * *χωρίς sampling* έχουμε τα καλύτερα αποτελέσματα με **total cost = 102**.
  * με *undersampling* πετυχαίνουμε χειρότερα αποτελέσματα σε σχέση με την περίπτωση που δεν εφαρμόζουμε καθόλου την τεχνική του sampling.
  * με *οversampling* έχουμε τα χειρότερα αποτελέσματα για τον συγκεκριμένο αλγόριρθμο 

3. Για το **Naive Bayes**:
  * *χωρίς sampling* έχουμε τα καλύτερα αποτελέσματα με **total cost = 190**.
  * με *undersampling* πετυχαίνουμε χειρότερα αποτελέσματα σε σχέση με την περίπτωση που δεν εφαρμόζουμε καθόλου την τεχνική του sampling.
  * με *οversampling* έχουμε τα χειρότερα αποτελέσματα για τον συγκεκριμένο αλγόριρθμο.

Συνολικά, παρατηρούμε ότι:
* στην προκειμένη περίπτωση **η μέθοδος sampling (τόσο το oversampling όοο και το undersampling) *χειροτερεύουν* την επίδοση των παραπάνω αλγορίθμων**
* η σειρά με την οποία παρουσίασα τους παραπάνω αλγορίθμους είναι και η σειρά κατάταξής τους στο συγκεκειμένο task και dataset.

# 3. Weighting

In [ ]:
X_train_val_df = pd.concat([X_train_df, X_val_df])
y_train_val_df = pd.concat([y_train_df, y_val_df])

In [ ]:
y_train_val_df.value_counts()

-1.0    561
 1.0    239
Name: Target, dtype: int64

In [ ]:
# create the cost matrix
cost_matrix = [[0, 1],
               [5, 0]]

classifiers = {
    'random forest': RandomForestClassifier(n_estimators=100, random_state=0),
    'linear SVM': SVC(kernel='linear', probability=True),
    'naive bayes': GaussianNB(),
}

y_test = y_test_df.to_numpy(dtype='int64')

for name in classifiers.keys():
  base_clf = classifiers[name]
  print(f'======================== {name} ========================')

  print("\n***** WITHOUT weights *****")
  base_clf.fit(X_train_val_df, y_train_val_df)
  y_preds = base_clf.predict(X_test_df)
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

  print("\n***** WITH weights *****")
  weights = np.zeros(y_train_val_df.shape[0])
  weights[y_train_val_df == -1] = 5
  weights[y_train_val_df == 1] = 1
  base_clf.fit(X_train_val_df, y_train_val_df, weights)
  y_preds = base_clf.predict(X_test_df)
  print(classification_report(y_test, y_preds))
  conf_m = confusion_matrix(y_test, y_preds).T
  print(conf_m)
  total_cost = np.sum(conf_m * cost_matrix)
  print(f"\nTotal Cost = {total_cost}")

======================== random forest ========================

***** WITHOUT weights *****
              precision    recall  f1-score   support

          -1       0.78      0.92      0.84       139
           1       0.69      0.39      0.50        61

    accuracy                           0.76       200
   macro avg       0.73      0.66      0.67       200
weighted avg       0.75      0.76      0.74       200

[[128  37]
 [ 11  24]]

Total Cost = 92

***** WITH weights *****
              precision    recall  f1-score   support

          -1       0.78      0.90      0.83       139
           1       0.64      0.41      0.50        61

    accuracy                           0.75       200
   macro avg       0.71      0.65      0.67       200
weighted avg       0.74      0.75      0.73       200

[[125  36]
 [ 14  25]]

Total Cost = 106
======================== linear SVM ========================

***** WITHOUT weights *****
              precision    recall  f1-score   support

 

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Από τα παραπάνω αποτελέσματα παρατηρώ τα παρακάτω:
1. Για το **Random Forest**:
  * *χωρίς weights* έχουμε καλύτερα αποτελέσματα (**total cost = 92**) σε σχέση με την περίπτωση όπου *'εχουμε weights* (**total cost = 102**).

2. Για το **Linear SVM**:
  * *χωρίς weights* έχουμε χειρότερα αποτελέσματα (**total cost = 102**) σε σχέση με την περίπτωση όπου *'εχουμε weights* (**total cost = 61**).

3. Για το **Naive Bayes**:
  * *χωρίς weights* έχουμε χειρότερα αποτελέσματα (**total cost = 190**) σε σχέση με την περίπτωση όπου *'εχουμε weights* (**total cost = 120**).

Συνολικά, παρατηρούμε ότι:
* στην προκειμένη περίπτωση **η χρήση των weights *βελτιώνουν* την επίδοση των αλγορίθμων Linear SVM και Naive Bayes**, ενώ ** *χειροτερεύουν* την επίδοση του αλγορίθμου Random Forest**
* στο συγκεκειμένο task και dataset, τα **καλύτερα αποτελέσματα** πετυχαίνει το μοντέλο **Linear SVM**, όταν εφαρμόζονται τα βάρη.
* στο συγκεκειμένο task και dataset, τη **μεγαλύτερη βελτίωση όταν εφαρμόζεται η χρήση των weights** πετυχαίνει το μοντέλο **Naive Bayes**.